In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import gc
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from gensim.models import LdaModel
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import mean_squared_erro

In [ ]:
df_train =pd.read_csv('/home/kesci/input/bytedance/train_final.csv',names=['qId','q','aId','a','target'],nrows=50000000)
df_test=pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv',names=['qId','q','aId','a'])

df_test['a']=df_test['a'].apply(lambda x:x[:-1])

In [ ]:
doc=pd.concat([df_train['q'],df_train['a'],df_test['q'],df_test['a']])
print(len(doc))
doc=pd.DataFrame(doc,columns=['docs'])
doc=doc.drop_duplicates(["docs"],keep="first")
tfidf = TfidfVectorizer(min_df=8)
tfidf.fit(doc["docs"])
del doc
gc.collect()

In [ ]:
m_q1_tf=tfidf.transform(pd.Series(df_train['q'].tolist()))
m_q2_tf=tfidf.transform(pd.Series(df_train['a'].tolist()))
e_q1_tf=tfidf.transform(pd.Series(df_test['q'].tolist()))
e_q2_tf=tfidf.transform(pd.Series(df_test['a'].tolist()))

In [ ]:
from scipy import sparse
test_feature=sparse.csc_matrix(sparse.hstack((e_q1_tf, e_q2_tf)))
del tfidf,df_test
gc.collect()
del e_q1_tf,e_q2_tf
gc.collect()
target=df_train['target']
del df_train
gc.collect()
train_feature=sparse.csc_matrix(sparse.hstack((m_q1_tf, m_q2_tf)))
del m_q1_tf,m_q2_tf
gc.collect()

In [ ]:
def sgdstacking():
    gc.collect()
    print('sgd stacking')
    df_stack=pd.DataFrame()
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    n_folds=5
    stack_train = np.zeros([train_feature.shape[0], 2])
    stack_test = np.zeros([test_feature.shape[0], 2])
    score_va=0
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (tr, va) in enumerate(folds.split(target.values)):
        print('stack:%d/%d' % ((i + 1), n_folds))
        sgd = SGDClassifier(random_state=1017, 
                            verbose=1,
                            loss='log',
                            penalty='elasticnet',
                            shuffle=True,
                            n_jobs=3,
                            l1_ratio=0.0001,
                            alpha=1e-05,
                            class_weight=None)
        gc.collect()
        sgd.fit(train_feature[tr], target.iloc[tr])
        score_va = sgd.predict_proba(train_feature[va])
        score_te = sgd.predict_proba(test_feature)
        print('得分' + str(mean_squared_error(target.iloc[va], sgd.predict(train_feature[va]))))
        stack_train[va] = score_va
        stack_test += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfC_sgd_{}'.format(i)] = stack[:, i]
    print('sgd done')
    return df_stack

In [ ]:
def ridgestacking():
    gc.collect()
    print('Ridge stacking')
    df_stack=pd.DataFrame()
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    n_folds=5
    stack_train = np.zeros([train_feature.shape[0], 2])
    stack_test = np.zeros([test_feature.shape[0], 2])
    score_va=0
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (tr, va) in enumerate(folds.split(target.values)):
        print('stack:%d/%d' % ((i + 1), n_folds))
        ridge = RidgeClassifier(random_state=1017,solver='sag',)
        ridge.fit(train_feature[tr], target.iloc[tr])
        score_va = ridge._predict_proba_lr(train_feature[va])
        score_te = ridge._predict_proba_lr(test_feature)
        print('得分' + str(mean_squared_error(target.iloc[va], ridge.predict(train_feature[va]))))
        stack_train[va] += score_va
        stack_test += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfV_ridge_{}'.format(i)] = stack[:, i]
    print('ridge done')
    return df_stack

In [ ]:
sgd_stack=sgdstacking()
ridgestacking=ridgestacking()
result=pd.concat([ridgestacking,sgd_stack],axis=1)
result.drop(['tfV_ridge_0','tfC_sgd_0'],axis=1,inplace=True)
train=result.iloc[0:50000000,:]
test=result.iloc[50000000:,:]

In [ ]:
train.to_csv('/home/kesci/work/train_feature_oof_final.csv',index=False)
test.to_csv('/home/kesci/work/test_feature_oof_final.csv',index=False)